In [ ]:
%pip install langdetect

In [16]:
import re
import sys
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer 

from langdetect import detect

from parapply import parapply

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import fasttext

import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv(sys.path[0]+'/train.csv',sep=',',dtype=object)
data.shape

(497819, 4)

In [2]:
def preprocessing(data,n_jobs = 1):

    for character in string.punctuation:
        data['name_1'] = data['name_1'].apply(lambda x: str(x).replace(character, ''))
        data['name_2'] = data['name_2'].apply(lambda x: str(x).replace(character, ''))

    data['lang_1'] = parapply(data['name_1'],lambda x: detect(x),n_jobs=n_jobs)
    data['lang_2'] = parapply(data['name_2'],lambda x: detect(x),n_jobs=n_jobs)

    lang_ru_indexes =data[data['lang_1']=='ru'].index
    print(lang_ru_indexes)
    if len(list(lang_ru_indexes))==0:
        lang_other_indexes = data.index
    else:
        lang_other_indexes =data.index.drop(lang_ru_indexes)
    stemmer_ru = SnowballStemmer("russian")
    stemmer_other = SnowballStemmer("english")

    if len(list(lang_ru_indexes))!=0:
        data.loc[lang_ru_indexes,'name_1_stemmed'] = parapply(data.loc[lang_ru_indexes,'name_1'],lambda x: stemmer_ru.stem(x),n_jobs=4)
        data.loc[lang_other_indexes,'name_1_stemmed'] = parapply(data.loc[lang_other_indexes,'name_1'],lambda x: stemmer_other.stem(x),n_jobs=4)
    else:
        data.loc[lang_other_indexes,'name_1_stemmed'] = parapply(data.loc[lang_other_indexes,'name_1'],lambda x: stemmer_other.stem(x),n_jobs=4)

    lang_ru_indexes =data[data['lang_2']=='ru'].index
    print(lang_ru_indexes)
    if len(list(lang_ru_indexes))==0:
        lang_other_indexes = data.index
    else:
        lang_other_indexes =data.index.drop(lang_ru_indexes)
    stemmer_ru = SnowballStemmer("russian")
    stemmer_other = SnowballStemmer("english")

    if len(list(lang_ru_indexes))!=0:
        data.loc[lang_ru_indexes,'name_2_stemmed'] = parapply(data.loc[lang_ru_indexes,'name_2'],lambda x: stemmer_ru.stem(x),n_jobs=n_jobs)
        data.loc[lang_other_indexes,'name_2_stemmed'] = parapply(data.loc[lang_other_indexes,'name_2'],lambda x: stemmer_other.stem(x),n_jobs=n_jobs)
    else:
        data.loc[lang_other_indexes,'name_2_stemmed'] = parapply(data.loc[lang_other_indexes,'name_2'],lambda x: stemmer_other.stem(x),n_jobs=n_jobs)
   
    
    return data


def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [3]:
dataset = preprocessing(data,4)

KeyboardInterrupt: 

In [15]:
dataset.to_csv(sys.path[0]+'/CompanyNamesSimularity_dataset.csv',sep=';',index=False)

In [4]:
dataset = pd.read_csv(sys.path[0]+'/CompanyNamesSimularity_dataset.csv',sep=';',dtype=object)

In [18]:
model_1 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=300,epoch=5, lr=0.1,thread=10)
model_2 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=150,epoch=7, lr=0.05,thread=10)
model_3 = fasttext.train_unsupervised(input=sys.path[0]+'/train_strings.csv',model="skipgram", minn=2, maxn=5, dim=100,epoch=10, lr=0.01,thread=10)

Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:   90969 lr:  0.000000 avg.loss:  0.554888 ETA:   0h 0m 0s  7.5% words/sec/thread:   92970 lr:  0.092471 avg.loss:  1.638084 ETA:   0h 0m23s 17.5% words/sec/thread:   91275 lr:  0.082497 avg.loss:  1.121155 ETA:   0h 0m21s 19.5% words/sec/thread:   91576 lr:  0.080489 avg.loss:  1.068077 ETA:   0h 0m20s 29.0% words/sec/thread:   92016 lr:  0.070988 avg.loss:  0.908398 ETA:   0h 0m18s 40.6% words/sec/thread:   91545 lr:  0.059436 avg.loss:  0.800276 ETA:   0h 0m15s  0h 0m11s
Read 4M words
Number of words:  16376
Number of labels: 0
Progress: 100.0% words/sec/thread:  153554 lr:  0.000000 avg.loss:  0.553253 ETA:   0h 0m 0s 53.8% words/sec/thread:  159130 lr:  0.023119 avg.loss:  0.715448 ETA:   0h 0m 9s 69.5% words/sec/thread:  156309 lr:  0.015272 avg.loss:  0.634221 ETA:   0h 0m 6s 79.9% words/sec/thread:  155292 lr:  0.010065 avg.loss:  0.599934 ETA:   0h 0m 4s 95.3% words/sec/thread:  154244 

In [5]:
model_1.save_model(sys.path[0]+'/model_1.bin')
model_2.save_model(sys.path[0]+'/model_2.bin')
model_3.save_model(sys.path[0]+'/model_3.bin')


In [5]:
model_1 = fasttext.load_model(sys.path[0]+'/model_1.bin')

In [7]:
dataset['is_duplicate'] = dataset['is_duplicate'].astype(int)

In [9]:
dataset_0 = dataset[dataset['is_duplicate']==0].sample(n=3600)
dataset_1 = dataset[dataset['is_duplicate']==1].sample(n=3600)

dataset_train_test = dataset_0.append(dataset_1).reset_index(drop=True)

print(dataset_train_test.shape)


(7200, 8)


In [10]:
embeddings_name_1 = np.zeros((dataset_train_test.shape[0],300),dtype='float32')
embeddings_name_2 = np.zeros((dataset_train_test.shape[0],300),dtype='float32')

for i,sentence in enumerate(dataset_train_test['name_1_stemmed']):
    embeddings_name_1[i,:] = model_1.get_sentence_vector(sentence)
for i,sentence in enumerate(dataset_train_test['name_2_stemmed']):
    embeddings_name_2[i,:] = model_1.get_sentence_vector(sentence)



In [11]:
embeddings_1 = np.hstack((embeddings_name_1,embeddings_name_2))
embeddings_1.shape

(7200, 600)

In [12]:
def simularity(embeddings):
    distances = np.zeros(embeddings.shape[0])

    for i, sentence in enumerate(embeddings):
        vector_i = sentence[0:300]
        vector_j = sentence[300:]
        distances[i] = cos_sim(vector_i,vector_j)

    return distances

cos_distance = simularity(embeddings_1)

dataset_train_test['cos_distance'] = cos_distance

dataset_train_test.head(10)

,pair_id,name_1,name_2,is_duplicate,lang_1,lang_2,name_1_stemmed,name_2_stemmed,cos_distance
0,469360,I B International,Guangzhou City Banger Fu Shoes Co Ltd,0,en,en,i b intern,guangzhou city banger fu shoes co ltd,0.445330
1,462512,Anchor Logistics,Eastway Logistics Usa Llc,0,en,en,anchor logist,eastway logistics usa llc,0.501545
2,412189,Fast Rite International Inc,Faj International,0,en,en,fast rite international inc,faj intern,0.534506
3,66926,Textiles Omnes SA,Shohagpur Textile Mills Ltd,0,fr,en,textiles omnes sa,shohagpur textile mills ltd,0.511297
4,219893,Russell Corp Aus PL,Acs International Sa De Cv,0,en,en,russell corp aus pl,acs international sa de cv,0.432505
5,422155,Sika S A,Ultra Plastik,0,tl,et,sika s a,ultra plastik,0.465515
6,438135,Samsung India Electronics Pvt Ltd,AG India Retail Pvt Ltd,0,en,id,samsung india electronics pvt ltd,ag india retail pvt ltd,0.843018
7,349072,Репсол,TERMOPRENE POLYMERS,0,mk,vi,репсол,termoprene polym,0.683315
8,172252,Guangzhou Jian Shun Metal Products Co Ltd,Texmate Products Co Ltd,0,en,en,guangzhou jian shun metal products co ltd,texmate products co ltd,0.839532
9,91962,PROBIGALP,Total Petrochemicals France SA,0,tl,en,probigalp,total petrochemicals france sa,0.547178


In [13]:
cols = ['emb_'+str(i) for i in range(embeddings_1.shape[1])]
embeddings_1_pd = pd.DataFrame(data=embeddings_1,columns=cols)
embeddings_1_pd.head(10)

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_590,emb_591,emb_592,emb_593,emb_594,emb_595,emb_596,emb_597,emb_598,emb_599
0,-0.001832,-0.005357,-0.018089,-0.037893,0.061595,0.016472,0.015916,-0.078075,0.054644,0.051844,...,-0.035557,-0.040607,0.007681,-0.033571,-0.054104,-0.006219,0.032292,-0.034382,-0.031298,-0.072496
1,0.007306,-0.010997,-0.048597,-0.007986,0.030631,-0.004835,0.032116,-0.047467,0.043619,-0.044867,...,-0.009196,-0.048925,0.014472,-0.068814,-0.049209,-0.038378,0.017818,0.000500,-0.002472,-0.010693
2,-0.039517,-0.015658,-0.010704,0.003947,-0.033919,0.074195,0.016669,-0.072259,0.079542,0.031767,...,-0.007273,-0.084578,0.029172,0.029100,-0.063051,-0.060403,0.026766,-0.074181,0.049591,-0.045154
3,-0.033255,-0.054866,-0.024194,0.010445,0.007902,0.084975,0.066347,-0.105140,0.080603,0.057089,...,-0.023507,-0.048227,0.010708,-0.018319,-0.019748,-0.022851,-0.011163,-0.029804,-0.046149,-0.029158
4,-0.009719,0.035695,-0.069145,-0.004493,0.010645,0.036689,-0.038998,-0.001504,0.080446,0.027402,...,0.013631,0.012361,0.017501,0.041185,-0.020170,-0.036760,0.015192,-0.011975,0.018698,-0.005453
5,-0.012938,0.000636,-0.044147,-0.020030,-0.022612,0.051612,0.056126,-0.087974,0.078466,-0.047529,...,0.013546,-0.084729,-0.034007,-0.056249,-0.074073,-0.040985,0.019617,-0.022484,0.040190,-0.038749
6,-0.061584,-0.051184,-0.081450,0.035143,0.014622,0.076482,-0.001165,-0.093878,0.069112,0.078263,...,-0.100199,-0.047080,0.016594,-0.018604,-0.043540,-0.031729,-0.003587,0.010222,0.021703,0.008686
7,-0.016338,0.027901,-0.089981,-0.012576,0.056740,0.077999,0.052903,-0.074421,0.057563,-0.008481,...,0.020491,-0.080450,-0.031769,-0.019681,-0.045459,0.024883,-0.036445,-0.048044,0.007728,0.025275
8,0.013925,-0.032157,-0.049879,0.012025,0.016370,0.076424,-0.011697,-0.060263,0.052594,0.014901,...,-0.062435,-0.038717,-0.047688,-0.032733,-0.020969,-0.008708,0.015790,-0.011955,-0.006279,-0.020582
9,-0.002780,0.019226,-0.093982,-0.004623,0.020593,0.048343,0.071213,-0.091670,-0.023990,-0.008614,...,-0.013756,-0.031798,-0.003775,-0.016691,-0.059477,-0.046844,-0.006594,-0.057879,0.028604,0.046187


In [14]:
dataset_train_test.columns

Index(['pair_id', 'name_1', 'name_2', 'is_duplicate', 'lang_1', 'lang_2',
       'name_1_stemmed', 'name_2_stemmed', 'cos_distance'],
      dtype='object')

In [15]:
dataset_train_test = dataset_train_test[['pair_id', 'name_1', 'name_2','cos_distance','is_duplicate']]
dataset_train_test = pd.concat((dataset_train_test,embeddings_1_pd),axis=1)

dataset_train_test.shape

(7200, 605)

In [20]:
dataset_train_test.drop(['pair_id', 'name_1', 'name_2','is_duplicate'],axis=1)

,cos_distance,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_590,emb_591,emb_592,emb_593,emb_594,emb_595,emb_596,emb_597,emb_598,emb_599
0,0.445330,-0.001832,-0.005357,-0.018089,-0.037893,0.061595,0.016472,0.015916,-0.078075,0.054644,...,-0.035557,-0.040607,0.007681,-0.033571,-0.054104,-0.006219,0.032292,-0.034382,-0.031298,-0.072496
1,0.501545,0.007306,-0.010997,-0.048597,-0.007986,0.030631,-0.004835,0.032116,-0.047467,0.043619,...,-0.009196,-0.048925,0.014472,-0.068814,-0.049209,-0.038378,0.017818,0.000500,-0.002472,-0.010693
2,0.534506,-0.039517,-0.015658,-0.010704,0.003947,-0.033919,0.074195,0.016669,-0.072259,0.079542,...,-0.007273,-0.084578,0.029172,0.029100,-0.063051,-0.060403,0.026766,-0.074181,0.049591,-0.045154
3,0.511297,-0.033255,-0.054866,-0.024194,0.010445,0.007902,0.084975,0.066347,-0.105140,0.080603,...,-0.023507,-0.048227,0.010708,-0.018319,-0.019748,-0.022851,-0.011163,-0.029804,-0.046149,-0.029158
4,0.432505,-0.009719,0.035695,-0.069145,-0.004493,0.010645,0.036689,-0.038998,-0.001504,0.080446,...,0.013631,0.012361,0.017501,0.041185,-0.020170,-0.036760,0.015192,-0.011975,0.018698,-0.005453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,0.430169,-0.073264,-0.015716,-0.099982,0.019117,0.058611,0.150183,0.004204,-0.121214,0.082927,...,-0.061353,-0.035592,0.003717,-0.050721,0.000943,-0.012953,0.006160,-0.022435,0.036606,0.001171
7196,0.777375,0.037517,0.019593,-0.045194,0.013140,0.084254,0.039886,-0.004177,-0.036135,0.024022,...,-0.013701,-0.084223,-0.012969,0.009103,-0.058086,-0.045581,0.013086,-0.094162,0.019973,0.024804
7197,0.943115,-0.044412,0.006240,-0.050735,0.031415,0.009828,0.059827,-0.009289,-0.008394,0.022051,...,-0.044809,-0.074404,-0.062049,-0.010760,-0.064863,0.010117,0.006718,-0.007186,0.044937,0.036848
7198,0.816286,-0.070342,-0.029253,-0.086381,0.021393,0.041321,0.105658,0.022926,-0.053025,0.057728,...,-0.029877,-0.067691,-0.004638,-0.027852,-0.058365,-0.011397,0.005090,-0.023680,0.073649,-0.024641


In [21]:
dataset_train_test['is_duplicate']

0       0
1       0
2       0
3       0
4       0
       ..
7195    1
7196    1
7197    1
7198    1
7199    1
Name: is_duplicate, Length: 7200, dtype: int64

In [ ]:
dataset_train_test.to_csv('dataset_train_test.csv', sep=';', index=False)

In [24]:
train, test, y_train, y_test = train_test_split(dataset_train_test.drop(['pair_id', 'name_1', 'name_2','is_duplicate'],axis=1),dataset_train_test['is_duplicate'],
test_size=0.3,shuffle=True,random_state=42)

In [25]:
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression()
logreg.fit(train,y_train)
y_train_pred = logreg.predict(train)

print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2513
           1       0.91      0.92      0.92      2527

    accuracy                           0.92      5040
   macro avg       0.92      0.92      0.92      5040
weighted avg       0.92      0.92      0.92      5040



In [26]:
y_test_pred = logreg.predict(test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1087
           1       0.91      0.93      0.92      1073

    accuracy                           0.92      2160
   macro avg       0.92      0.92      0.92      2160
weighted avg       0.92      0.92      0.92      2160



In [29]:
model_2 = fasttext.load_model(sys.path[0]+'/model_2.bin')

dataset_train_test = dataset_0.append(dataset_1).reset_index(drop=True)

embeddings_name_1 = np.zeros((dataset_train_test.shape[0],300),dtype='float32')
embeddings_name_2 = np.zeros((dataset_train_test.shape[0],300),dtype='float32')

for i,sentence in enumerate(dataset_train_test['name_1_stemmed']):
    embeddings_name_1[i,:] = model_1.get_sentence_vector(sentence)
for i,sentence in enumerate(dataset_train_test['name_2_stemmed']):
    embeddings_name_2[i,:] = model_1.get_sentence_vector(sentence)

embeddings_2 = np.hstack((embeddings_name_1,embeddings_name_2))



cos_distance = simularity(embeddings_2)

dataset_train_test['cos_distance'] = cos_distance

cols = ['emb_'+str(i) for i in range(embeddings_2.shape[1])]
embeddings_2_pd = pd.DataFrame(data=embeddings_2,columns=cols)

dataset_train_test = dataset_train_test[['pair_id', 'name_1', 'name_2','cos_distance','is_duplicate']]
dataset_train_test = pd.concat((dataset_train_test,embeddings_2_pd),axis=1)

train, test, y_train, y_test = train_test_split(dataset_train_test.drop(['pair_id', 'name_1', 'name_2','is_duplicate'],axis=1),dataset_train_test['is_duplicate'],
test_size=0.3,shuffle=True,random_state=42)


logreg = LogisticRegression()
logreg.fit(train,y_train)

y_train_pred = logreg.predict(train)

print(classification_report(y_train, y_train_pred))

y_test_pred = logreg.predict(test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2513
           1       0.91      0.92      0.92      2527

    accuracy                           0.92      5040
   macro avg       0.92      0.92      0.92      5040
weighted avg       0.92      0.92      0.92      5040

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1087
           1       0.91      0.93      0.92      1073

    accuracy                           0.92      2160
   macro avg       0.92      0.92      0.92      2160
weighted avg       0.92      0.92      0.92      2160



In [33]:
model_3 = fasttext.load_model(sys.path[0]+'/model_3.bin')

dataset_train_test = dataset_0.append(dataset_1).reset_index(drop=True)

embeddings_name_1 = np.zeros((dataset_train_test.shape[0],300),dtype='float32')
embeddings_name_3 = np.zeros((dataset_train_test.shape[0],300),dtype='float32')

for i,sentence in enumerate(dataset_train_test['name_1_stemmed']):
    embeddings_name_1[i,:] = model_1.get_sentence_vector(sentence)
for i,sentence in enumerate(dataset_train_test['name_2_stemmed']):
    embeddings_name_3[i,:] = model_1.get_sentence_vector(sentence)

embeddings_3 = np.hstack((embeddings_name_1,embeddings_name_3))



cos_distance = simularity(embeddings_3)

dataset_train_test['cos_distance'] = cos_distance

cols = ['emb_'+str(i) for i in range(embeddings_3.shape[1])]
embeddings_3_pd = pd.DataFrame(data=embeddings_3,columns=cols)

dataset_train_test = dataset_train_test[['pair_id', 'name_1', 'name_2','cos_distance','is_duplicate']]
dataset_train_test = pd.concat((dataset_train_test,embeddings_3_pd),axis=1)

train, test, y_train, y_test = train_test_split(dataset_train_test.drop(['pair_id', 'name_1', 'name_2','is_duplicate'],axis=1),dataset_train_test['is_duplicate'],
test_size=0.3,shuffle=True,random_state=42)


logreg = LogisticRegression()
logreg.fit(train,y_train)

y_train_pred = logreg.predict(train)

print(classification_report(y_train, y_train_pred))

y_test_pred = logreg.predict(test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2513
           1       0.91      0.92      0.92      2527

    accuracy                           0.92      5040
   macro avg       0.92      0.92      0.92      5040
weighted avg       0.92      0.92      0.92      5040

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1087
           1       0.91      0.93      0.92      1073

    accuracy                           0.92      2160
   macro avg       0.92      0.92      0.92      2160
weighted avg       0.92      0.92      0.92      2160

